In [2]:
import errno
import json
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import helper
import argparse

%matplotlib inline

In [3]:
# Some useful constants
DRIVING_LOG_FILE = os.getcwd() + os.sep + os.pardir + '/datasets/2/driving_log.csv'
IMG_PATH = os.getcwd() + os.sep + os.pardir
WINDOW_SIZE = 7
OUTPUT_DIR = os.getcwd() + os.sep + os.pardir + '/flow_%d_right' % WINDOW_SIZE
os.makedirs(OUTPUT_DIR)

In [4]:
data_frame = pd.read_csv(DRIVING_LOG_FILE, usecols=[0,1,2,3])

In [5]:
last = []
prev_image = None

for i in range(1, len(data_frame)):
    img = cv2.imread(IMG_PATH + "/datasets/2/DataRight/" + str(i) + '.jpg')
    prev = prev_image if prev_image is not None else img
    prev_image = img
    prev = cv2.cvtColor(prev, cv2.COLOR_RGB2GRAY)
    next = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    last.append(flow)
    if len(last) > WINDOW_SIZE: 
        last.pop(0)
    avg_flow = sum(last) / len(last)
    mag, ang = cv2.cartToPolar(avg_flow[..., 0], avg_flow[..., 1])

    hsv = np.zeros_like(prev_image)
    hsv[..., 1] = 255
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cv2.imwrite(os.path.join(OUTPUT_DIR, str(i) + ".jpg"), bgr)